## 전처리

In [22]:
from describe import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
#데이터 불러오기
all_data = pd.read_csv("data/all_data.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_21852\480390161.py:2: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("data/all_data.csv")


In [24]:
# target 값이 비어있지 않은 데이터(Train data)
train_data = all_data[all_data['target'].notnull()]
train_data = train_data.drop(['Set ID'], axis=1)    

# target 값이 비어있는 데이터(Test data)
test_data = all_data[all_data['target'].isnull()]

# 결과 확인
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")

Train Data Shape: (40506, 60)
Test Data Shape: (17361, 61)


In [25]:
# 각 열의 결측값 개수 확인
missing_values = train_data.isnull().sum()

# 결측값 비율 계산
missing_percentage = (missing_values / len(train_data)) * 100

# 결과를 데이터프레임으로 변환하여 보기 편하게 출력
missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
print(missing_info[missing_info['Missing Values'] > 0])

                                                   Missing Values  \
DISCHARGED VpS RESIN(Stage1) Collect Result_Fill1               1   
DISCHARGED VpS RESIN(Stage2) Collect Result_Fill1               1   
DISCHARGED VpS RESIN(Stage3) Collect Result_Fill1               1   

                                                   Percentage (%)  
DISCHARGED VpS RESIN(Stage1) Collect Result_Fill1        0.002469  
DISCHARGED VpS RESIN(Stage2) Collect Result_Fill1        0.002469  
DISCHARGED VpS RESIN(Stage3) Collect Result_Fill1        0.002469  


In [26]:
# train_data에서 결측값이 있는 행의 개수 확인
missing_rows_count = train_data.isnull().any(axis=1).sum()

print(f"결측값이 있는 행의 개수: {missing_rows_count}")

결측값이 있는 행의 개수: 1


In [27]:
# 결측값이 있는 행 삭제
train_data_cleaned = train_data.dropna()

# 결과 확인
print(f"원래 Train Data Shape: {train_data.shape}")
print(f"클린된 Train Data Shape: {train_data_cleaned.shape}")


"""
결측값 처리 결과
    결측값 발생 행의 개수는 1개였으므로 제거하였다.
    결측값이 있는 행을 제거한 후의 Train Data Shape는 (40505, 60)이다.(Set ID 열도 제거하였음)
"""


원래 Train Data Shape: (40506, 60)
클린된 Train Data Shape: (40505, 60)


'\n결측값 처리 결과\n    결측값 발생 행의 개수는 1개였으므로 제거하였다.\n    결측값이 있는 행을 제거한 후의 Train Data Shape는 (40505, 60)이다.(Set ID 열도 제거하였음)\n'

In [28]:
y = train_data_cleaned['target']
train_data_cleaned = train_data_cleaned.drop(['target'], axis=1)

### 범주형 데이터 인코딩 - 정규화

In [29]:
### 1단계: 범주형 데이터 인코딩
from sklearn.preprocessing import LabelEncoder

# 범주형 변수를 인코딩하는 함수
def encode_categorical_features(data):
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))  # 문자열로 변환 후 인코딩
        label_encoders[column] = le
    return data, label_encoders

# Train data 인코딩
train_data_encoded, label_encoders = encode_categorical_features(train_data_cleaned) #이후 Inversion Transform 메서드를 사용하기 위해 label_encoders를 저장



In [30]:
train_data_encoded.shape

(40505, 59)

In [31]:
from sklearn.preprocessing import MinMaxScaler

# 정규화 함수
def normalize_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.select_dtypes(include=[np.number]))  # 수치형 데이터만 정규화
    scaled_df = pd.DataFrame(scaled_data, columns=data.select_dtypes(include=[np.number]).columns, index=data.index)
    return scaled_df

# 정규화 진행
train_data_scaled = normalize_data(train_data_encoded)

# 원본 데이터와 결합 (인코딩된 범주형 데이터는 함께 포함)
train_data_final = pd.concat([train_data_scaled, train_data_encoded.select_dtypes(include=['object'])], axis=1)

In [32]:
#train_data_final csv 파일로 저장
train_data_final.to_csv("data/train_data_final.csv", index=False)
y.to_csv("data/y.csv", index=False)

## 데이터 가져오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


# 데이터프레임에서 특성과 레이블 나누기
X = pd.read_csv("./data/train_data_final.csv")  # 데이터 불러오기
y = pd.read_csv("./data/y.csv")  # 데이터 불러오기

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## 미니 배치 학습

In [ ]:
# 미니 배치 생성 함수 정의
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.array(X)
    labels = np.array(y)
    
    # 데이터와 레이블을 결합한 후 섞기
    combined = np.column_stack((data, labels))
    np.random.shuffle(combined)

    # 미니 배치 생성
    for i in range(0, len(combined), batch_size):
        mini_batch = combined[i:i + batch_size]
        mini_batches.append((mini_batch[:, :-1], mini_batch[:, -1]))  # 특성과 레이블 분리
        
    return mini_batches

# Random Forest 모델 정의
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 하이퍼파라미터 설정
batch_size = 32
epochs = 10

# 미니 배치 학습
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    mini_batches = create_mini_batches(X_train, y_train, batch_size)
    for X_mini_batch, y_mini_batch in mini_batches:
        model.fit(X_mini_batch, y_mini_batch)  # 각 미니 배치에 대해 학습 진행

In [19]:
# 최종 학습 후 전체 테스트 데이터에 대해 예측
y_pred = model.predict(X_test)


c:\lg_capstone\lg_capstone\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [33]:
# y_pred를 원래 레이블로 변환
y_pred_labels = label_encoders['target'].inverse_transform(y_pred)

KeyError: 'target'

In [34]:
# F1 스코어 평가
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.2f}')

ValueError: pos_label=1 is not a valid label. It should be one of ['AbNormal', 'Normal']